# Code

Below is some python code for manipulating Turing machines. Scroll below to the palindromes section to see how it is used.

In [1]:
import copy


def initialConf(machine, word):
    if "onlyWorkTape" in machine:
        return {
            "state": machine["initial"],
            "work tape": ["⊢"] + word,
            "work head": 0,
        }
    else:
        return {
            "state": machine["initial"],
            "input tape": ["⊢"] + word + ["⊣"],
            "input head": 0,
            "work tape": ["⊢"],
            "work head": 0,
        }


def doesTransitionApply(trans, conf):
    # Wrong state.
    if trans["state before"] != conf["state"]:
        return False

    # Questions about work tape are asked only if it exists.
    if "input tape" in conf:
        # Wrong input letter.
        if ("input letter before" in trans) and (
            trans["input letter before"] != conf["input tape"][conf["input head"]]
        ):
            return False
        # Input head falls out of input tape.
        if ("move input head" in trans) and (
            (conf["input head"] + trans["move input head"] >= len(conf["input tape"]))
            or (conf["input head"] + trans["move input head"] < 0)
        ):
            return False

    # Wrong work letter.
    if ("work letter before" in trans) and (
        trans["work letter before"] != (conf["work tape"])[conf["work head"]]
    ):
        return False

    # Work head falls out of work tape to the left.
    if ("move work head" in trans) and (
        trans["move work head"] + conf["work head"] < 0
    ):
        return False

    return True


# Applies a transitions and returns the new configuration.
def applyTransition(trans, conf):
    if not doesTransitionApply(trans, conf):
        raise Exception("tried to apply transition that does not apply")

    # A special configuration that halts and just gives the result, which is typically accept/reject.
    if "halt" in trans:
        return {"halt": trans["halt"]}

    newconf = copy.deepcopy(conf)

    newconf["state"] = trans["state after"]

    if "move input head" in trans:
        newconf["input head"] += trans["move input head"]

    if "work letter after" in trans:
        newconf["work tape"][newconf["work head"]] = trans["work letter after"]

    if "move work head" in trans:
        newconf["work head"] += trans["move work head"]

    # If the work head moved out of work tape, add a new blank symbol.
    if newconf["work head"] >= len(newconf["work tape"]):
        newconf["work tape"] += [""]

    return newconf


# Returns the list of all avaialable transitions.
def availableTransitions(machine, conf):
    retval = []

    if (conf == "accept") or (conf == "reject"):
        return retval

    for t in machine["transitions"]:
        if doesTransitionApply(t, conf):
            retval += [t]
    return retval


# Returns the list of all configurations in a run, for the given input string.
# If several transitions apply, the first one is used.
def run(machine, string):
    conf = initialConf(machine, string)

    # The return value is a list of configurations.
    retval = []
    while True:
        retval += [conf]

        if "halt" in conf:
            break

        # There is a timeout of 10k transitions.
        if len(retval) > 10_000:
            retval += [{"halt": "reject because run length exceeded"}]
            break

        transitionList = availableTransitions(machine, conf)

        if len(transitionList) == 0:
            retval += [{"halt": "reject because no transition can be applied"}]
            break

        # We use first available transition.
        t = transitionList[0]
        conf = applyTransition(t, conf)

    return retval


# This part of the code displays runs in HTML.
from IPython.core.display import HTML


# Returns HTML representation of a single cell.
def cellHTML(content, head):
    if head:
        return (
            '<span style="background-color:red; padding: 5px; margin: 3px">'
            + content
            + "</span>"
        )
    else:
        return (
            '<span style="background-color:lightgrey; padding: 5px; margin: 3px">'
            + content
            + "</span>"
        )


# Returns HTML representation of a tape, which can be the input or work tape.
def tapeHTML(string, head):
    if head < 0 or head >= len(string):
        raise Exception("head out of bounds")
    index = 0
    retval = '<span style="margin:20px">'
    for x in string:
        retval += cellHTML(x, index == head)
        index += 1
    return retval + "</span>"


# Returns HTML representation of an entire configuration.
def confHTML(conf):
    retval = '<div style="padding:20px">'
    if "halt" in conf:
        retval += conf["halt"]
    else:
        retval += (
            '<span style="display:inline-block; width:100px"> state:'
            + conf["state"]
            + "</span>"
        )
        # If the machine is two tape, then it has a separate input tape.
        if "input tape" in conf:
            retval += "input tape:"
            retval += tapeHTML(conf["input tape"], conf["input head"])
            retval += "work tape:"
        # Both one and two tape machines have a work tape.
        retval += tapeHTML(conf["work tape"], conf["work head"])

    retval += "</div>"
    return retval


def displayConf(conf):
    HTML(confHTML(conf))


def displayRun(machine, string):
    retval = ""
    for conf in run(machine, string):
        retval += confHTML(conf)
    return HTML(retval)

The syntax of Turing machines is explained below, on the example of a machine for palindromes

# Palindromes

We give two example machines for palindromes. One is the (default) two tape machine, and the other is a one tape machine.

### Two tape palindromes

Here is a (two tape) Turing machine with input alphabet ['a','b'] that accepts exactly the palindromes. 

In [2]:
machine = { 'initial': 'p',
            'transitions': [
              {'state before': 'p', 'state after': 'p', 'input letter before': '⊢', 'move input head': 1, 'move work head': 1, 'work letter after': '⊢'},
              {'state before': 'p', 'state after': 'p', 'input letter before': 'a', 'move input head': 1, 'move work head': 1, 'work letter after': 'a'},
              {'state before': 'p', 'state after': 'p', 'input letter before': 'b', 'move input head': 1, 'move work head': 1, 'work letter after': 'b'},
              {'state before': 'p', 'state after': 'q', 'input letter before': '⊣', 'move input head': -1},
              {'state before': 'q', 'state after': 'q', 'input letter before': 'a', 'move input head': -1},
              {'state before': 'q', 'state after': 'q', 'input letter before': 'b', 'move input head': -1},
              {'state before': 'q', 'state after': 'r', 'input letter before': '⊢', 'move input head': 1, 'move work head': -1},
              {'state before': 'r', 'state after': 'r', 'input letter before': 'a', 'work letter before': 'a', 'move work head': -1, 'move input head': 1},
              {'state before': 'r', 'state after': 'r', 'input letter before': 'b', 'work letter before': 'b', 'move work head': -1, 'move input head': 1},
              {'state before': 'r', 'input letter before': '⊣', 'halt': 'accept'},
            ]
          }

A machine consists of an initial state, and a list of transitions. The states are implicit – these are the states that appear in the transitions, and also the work and input alphabet are implicit. The states are strings (and not just letters), the same is true for letters of the input and work alphabet, i.e. each such letter is a string. A transition is a record that has the following fields: 'state before', 'state after', 'input letter before', 'work letter before', 'move input head', 'move work head', 'input letter after', 'work letter after'. The fields concerning the letters (both input and work, both before and after) are optional. Omitting a '* letter before' field means the transition applies to all letters, omitting a '* letter after' field means the cell keeps its old value and is not overwritten.

In [3]:
displayRun(machine, ["a", "b", "a"])

### One tape palindromes

Here is a one tape machine that recognizes the palindromes, again over the alphabet 'a' and 'b'. In the one tape variant, there is a flag 'onlyWorkTape' : true, which is not used in the default two tape format. The work tape is the only tape.

In [4]:
oneTapeMachine = {  'onlyWorkTape': True,
                    'initial': 'init',
                    'transitions': [
                      {'state before': 'init', 'state after': 'q', 'move work head': 1},
                      {'state before': 'q', 'state after': 'senda', 'work letter before': 'a', 'work letter after': '', 'move work head': 1},
                      {'state before': 'senda', 'state after': 'senda', 'work letter before': 'a', 'move work head': 1},
                      {'state before': 'senda', 'state after': 'senda', 'work letter before': 'b', 'move work head': 1},
                      {'state before': 'senda', 'state after': 'checka', 'work letter before': '', 'move work head': -1},
                      {'state before': 'checka', 'state after': 'return', 'work letter before': 'a', 'work letter after': '', 'move work head': -1},
                      {'state before': 'q', 'state after': 'sendb', 'work letter before': 'b', 'work letter after': '', 'move work head': 1},
                      {'state before': 'sendb', 'state after': 'sendb', 'work letter before': 'a', 'move work head': 1},
                      {'state before': 'sendb', 'state after': 'sendb', 'work letter before': 'b', 'move work head': 1},
                      {'state before': 'sendb', 'state after': 'checkb', 'work letter before': '', 'move work head': -1},
                      {'state before': 'checkb', 'state after': 'return', 'work letter before': 'b', 'work letter after': '', 'move work head': -1},
                      {'state before': 'return', 'state after': 'return', 'work letter before': 'a', 'move work head': -1},
                      {'state before': 'return', 'state after': 'return', 'work letter before': 'b', 'move work head': -1},
                      {'state before': 'return', 'state after': 'q', 'work letter before': '', 'move work head': 1},
                      {'state before': 'q', 'work letter before': '', 'halt': 'accept'},
                      {'state before': 'checka', 'work letter before': '', 'halt': 'accept'},
                      {'state before': 'checkb', 'work letter before': '', 'halt': 'accept'},
                    ]
                  }

In [5]:
displayRun(oneTapeMachine, ["a", "b", "b", "a"])

In [6]:
import random
import string


# Generates a random string of length 5.
def getRandomString():
    return "".join(random.choice(string.ascii_letters) for _ in range(5))


def getAlphabet(machine):
    res = set({"⊢", "⊣", ""})
    for trans in machine["transitions"]:
        for key in ["input letter before", "work letter before", "work letter after"]:
            if key in trans:
                res.add(trans[key])
    return list(res)


# Split transitions into buckets. Each bucket contains transitions that share "state before" and "input letter before".
def bucketTransitions(machine):
    buckets = {}
    for trans in machine["transitions"]:
        st = trans["state before"], trans.get("input letter before")
        if st in buckets:
            buckets[st].append(trans)
        else:
            buckets[st] = [trans]
    return buckets


def preprocessTransitions(machine):
    newTransitions = [
        {"state before": "preprocess", "state after": "preprocess1", "work letter before": "⊢", "work letter after": "⊢'", "move work head": 1},
        {"state before": "preprocess1", "state after": "preprocess2", "work letter before": "", "work letter after": "⊣", "move work head": 1},
        {"state before": "preprocess1", "state after": "preprocess1", "move work head": 1},
        {"state before": "preprocess2", "state after": "preprocess3", "work letter before": "", "work letter after": "⊢'", "move work head": -1},
        {"state before": "preprocess3", "state after": machine["initial"], "work letter before": "⊢'"},
        {"state before": "preprocess3", "state after": "preprocess3", "move work head": -1},
    ]
    return newTransitions


def createTransition(stateBefore, stateAfter=None, workBefore=None, workAfter=None, moveWorkHead=None, halt=None):
    trans = {"state before": stateBefore}
    if stateAfter is not None:
        trans["state after"] = stateAfter
    if workBefore is not None:
        trans["work letter before"] = workBefore
    if workAfter is not None:
        trans["work letter after"] = workAfter
    if moveWorkHead is not None:
        trans["move work head"] = moveWorkHead
    if halt is not None:
        trans["halt"] = halt
    return trans


def eliminateInputTape(machine):
    if "onlyWorkTape" in machine and machine["onlyWorkTape"]:
        return machine

    alphabet = getAlphabet(machine)
    markedAlphabet = [letter + "'" for letter in alphabet]

    # Array of new transitions. The idea is to have two tapes on one tape and mark both original input and work tapes with '.
    # Before starting with the initial state, we add a "preprocess" stage that simply creates a second tape on the initial
    # tape: ⊢inputword___ -> ⊢'inputword⊣⊢'___. We mark both ⊢ with '. They are the heads of input and work tapes, respectively.
    newTransitions = preprocessTransitions(machine)

    # For each transition we split it into few steps:
    # 1. Check input letter.
    # 2. If it matches go to the work tape.
    # 3. Check work letter.
    # 4. If it matches go back to the input tape and do the job there too.
    for bucket in bucketTransitions(machine).values():
        go_to_work_tape = getRandomString()

        for trans in bucket:
            name = getRandomString()
            do_job_work_tape = name + "1"
            go_to_input_tape = name + "2"
            do_job_input_tape = name + "3"

            # 1. Go to the work tape.
            if "input letter before" in trans:
                newTransitions.append(createTransition(trans["state before"], go_to_work_tape, moveWorkHead=1, workBefore=trans["input letter before"] + "'"))
            else:
                newTransitions.append(createTransition(trans["state before"], go_to_work_tape, moveWorkHead=1))

            for letter in alphabet:
                newTransitions.append(createTransition(go_to_work_tape, go_to_work_tape, workBefore=letter, moveWorkHead=1))

            # 2. Do the job in the work tape.
            if "work letter before" in trans:
                dest = trans["work letter before"] + "'"

                # Check move out of the work tape.
                if dest == "⊢'" and trans.get("move work head") == -1:
                    continue

                if "work letter after" in trans:
                    newTransitions.append(createTransition(go_to_work_tape, do_job_work_tape, workBefore=dest, workAfter=trans["work letter after"], moveWorkHead=trans.get("move work head")))
                else:
                    newTransitions.append(createTransition(go_to_work_tape, do_job_work_tape, workBefore=dest, workAfter=trans["work letter before"], moveWorkHead=trans.get("move work head")))
            else:
                for letter in markedAlphabet:
                    # Check move out of the work tape.
                    if letter == "⊢'" and trans.get("move work head") == -1:
                        continue

                    if "work letter after" in trans:
                        newTransitions.append(createTransition(go_to_work_tape, do_job_work_tape, workBefore=letter, workAfter=trans["work letter after"], moveWorkHead=trans.get("move work head")))
                    else:
                        newTransitions.append(createTransition(go_to_work_tape, do_job_work_tape, workBefore=letter, workAfter=letter[:-1], moveWorkHead=trans.get("move work head")))

            # Append ' to a new position in the work tape.
            for letter in alphabet:
                newTransitions.append(createTransition(do_job_work_tape, go_to_input_tape, workBefore=letter, workAfter=letter + "'", moveWorkHead=-1))

            # 2.5. Check if halt.
            if "halt" in trans:
                newTransitions.append(createTransition(go_to_input_tape, halt=trans["halt"]))
                continue

            # 4. Do the job on input tape.
            for letter in markedAlphabet:
                newTransitions.append(createTransition(go_to_input_tape, do_job_input_tape, workBefore=letter, workAfter=letter[:-1], moveWorkHead=trans.get("move input head")))

            # 3. Go back to the input tape.
            newTransitions.append(createTransition(go_to_input_tape, go_to_input_tape, moveWorkHead=-1))

            # Append ' to a new position in the input tape.
            for letter in alphabet:
                newTransitions.append(createTransition(do_job_input_tape, trans["state after"], workBefore=letter, workAfter=letter + "'"))

    return {
        "onlyWorkTape": True,
        "initial": "preprocess",
        "transitions": newTransitions,
    }

In [7]:
displayRun(eliminateInputTape(machine), ["a", "b", "a"])